# Lets save our data as a huggingface dataset, so it's quick to reuse



In [1]:
# import your package
%load_ext autoreload
%autoreload 2

from loguru import logger
import sys
logger.remove()
logger.add(sys.stderr, format="<level>{message}</level>", level="INFO")

import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
import numpy as np


from typing import Optional, List, Dict, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

import pickle
import hashlib
from pathlib import Path

import transformers
from datasets import Dataset, DatasetInfo, load_from_disk, load_dataset


from tqdm.auto import tqdm
import os, re, sys, collections, functools, itertools, json

transformers.__version__


'4.31.0'

In [4]:
from src.models.load import load_model
from src.datasets.load import ds2df
from src.datasets.load import rows_item
from src.datasets.batch import batch_hidden_states
from src.datasets.hs import get_choices_as_tokens, default_class2choices, choice2ids, scores2choice_probs


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ubuntu/mambaforge/envs/dlk3/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/ubuntu/mambaforge/envs/dlk3/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/ubuntu/mambaforge/envs/dlk3/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/ubuntu/mambaforge/envs/dlk3/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/ubuntu/mambaforge/envs/dlk3/lib/libcudart.so'), PosixPath('/home/ubuntu/mambaforge/envs/dlk3/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


# Params

In [5]:
# Params
BATCH_SIZE = 10  # None # None means auto # 6 gives 16Gb/25GB. where 10GB is the base model. so 6 is 6/15
USE_MCDROPOUT = True

from src.extraction.config import ExtractConfig

cfg = ExtractConfig(
    model="HuggingFaceH4/starchat-beta",
    datasets = ["imdb", "amazon_polarity", "truthful_qa",
                #"super_glue:boolq", "EleutherAI/truthful_qa_mc", "EleutherAI/arithmetic", "NeelNanda/counterfact-tracing"
                ],
    max_examples=(100, 101),
)
cfg

ExtractConfig(model='HuggingFaceH4/starchat-beta', datasets=['imdb', 'amazon_polarity', 'truthful_qa'], data_dirs=(), int4=True, max_examples=(100, 101), num_shots=2, num_variants=-1, layers=(), seed=42, token_loc='last', template_path=None)

In [6]:
import elk
elk

<module 'elk' from '/home/ubuntu/mambaforge/envs/dlk3/lib/python3.11/site-packages/elk/__init__.py'>

# Model

Chosing:
- https://old.reddit.com/r/LocalLLaMA/wiki/models
- https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
- https://github.com/deep-diver/LLM-As-Chatbot/blob/main/model_cards.json


A uncensored and large coding ones might be best for lying.

In [7]:
model, tokenizer = load_model(model_repo=cfg.model)

changing pad_token_id from None to 0
changing padding_side from right to left
changing truncation_side from right to left


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

GPTBigCodeConfig {
  "_name_or_path": "HuggingFaceH4/starchat-beta",
  "activation_function": "gelu",
  "architectures": [
    "GPTBigCodeForCausalLM"
  ],
  "attention_softmax_in_fp32": true,
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "inference_runner": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_batch_size": null,
  "max_sequence_length": null,
  "model_type": "gpt_bigcode",
  "multi_query": true,
  "n_embd": 6144,
  "n_head": 48,
  "n_inner": 24576,
  "n_layer": 40,
  "n_positions": 8192,
  "pad_key_length": true,
  "pre_allocate_kv_cache": false,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false
  },
  "resid_pdrop": 0.1,


# Load Dataset

NameError: name 'prompts' is not defined

In [65]:
from itertools import chain, islice
from datasets import Dataset
import functools
# from datasets.arrow_dataset import Dataset
from src.prompts.prompt_loading import load_prompts

@functools.lru_cache()
def count_tokens(s):
    return len(tokenizer(s).input_ids)

def answer_len(answer_choices: list):
    a = count_tokens(answer_choices[0])
    b = count_tokens(answer_choices[1])
    return max(a, b)


def sample_n_true_y_false_prompts(prompts, num_truth=1, num_lie=1, seed=42):
    """sample some truth and some false"""
    df = pd.DataFrame(prompts)
    
    # restrict to template where the choices are a single token
    m = df.answer_choices.map(answer_len)==1
    df = df[m]
    df = pd.concat([
        df.query("instructed_to_lie==True").sample(num_truth, random_state=seed),
        df.query("instructed_to_lie==False").sample(num_lie, random_state=seed)])
    return df.to_dict(orient="records")


# for ds_name in ds_names:
#     for split_type in ["train", "test"]:
        
# loop through all prompts in this dataset
ds_names = cfg.datasets
split_type = "train"

ds_name = ds_names[0]
prompt_ds = load_prompts(
    ds_name,
    num_shots=cfg.num_shots,
    split_type=split_type,
    template_path=cfg.template_path,
    seed=cfg.seed,
)

# for each example, sample true and false
N = cfg.max_examples[split_type!="train"]
g = map(lambda x: sample_n_true_y_false_prompts(x, seed=cfg.seed), prompt_ds)

# and combine them into one big list
g = chain.from_iterable(g) 
prompt_ds2 = list(tqdm(islice(g, N), total=N))

# convert to hugginface dataset
dataset = Dataset.from_list(prompt_ds2)
dataset

  0%|          | 0/100 [00:00<?, ?it/s]

Extracting 13 variants of each prompt


Dataset({
    features: ['answer', 'question', 'answer_choices', 'template_name', 'label_true', 'label_instructed', 'instructed_to_lie', 'sys_instr_name'],
    num_rows: 100
})

In [66]:
b = next(iter(prompt_ds))
# sample_n_true_y_false_prompts(b)

## Format prompts

The prompt is the thing we most often have to change and debug. So we do it explicitly here.

We do it as transforms on a huggingface dataset.

In this case we use multishot examples from train, and use the test set to generated the hidden states dataset. We will test generalisation on a whole new dataset.


In [67]:
ds = (
    dataset
    .map(
        lambda ex: tokenizer(
            ex["question"], padding="max_length", max_length=600, truncation=True, add_special_tokens=True,
            # return_tensors="pt",
            return_attention_mask=True,
        ),
        batched=True,
    )
    .map(
        lambda r: {"prompt_truncated": tokenizer.batch_decode(r["input_ids"])},
        batched=True,
    )
)
ds

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['answer', 'question', 'answer_choices', 'template_name', 'label_true', 'label_instructed', 'instructed_to_lie', 'sys_instr_name', 'input_ids', 'attention_mask', 'prompt_truncated'],
    num_rows: 100
})

## Save as Huggingface Dataset

In [68]:
# get dataset filename
sanitize = lambda s:s.replace('/', '').replace('-', '_') if s is not None else s

dataset_name = f"{sanitize(cfg.model)}_{ds_name}_{split_type}_{N}"
dataset_name
f = f"../.ds/{dataset_name}"
print(f)

../.ds/HuggingFaceH4starchat_beta_imdb_train_100


In [69]:
# ds[10]

In [70]:
gen_kwargs = dict(
    model=model,
    tokenizer=tokenizer,
    data=ds,
    batch_size=BATCH_SIZE,
)
gen_kwargs

{'model': GPTBigCodeForCausalLM(
   (transformer): GPTBigCodeModel(
     (wte): Embedding(49156, 6144)
     (wpe): Embedding(8192, 6144)
     (drop): Dropout(p=0.1, inplace=False)
     (h): ModuleList(
       (0-39): 40 x GPTBigCodeBlock(
         (ln_1): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
         (attn): GPTBigCodeAttention(
           (c_attn): Linear4bit(in_features=6144, out_features=6400, bias=True)
           (c_proj): Linear4bit(in_features=6144, out_features=6144, bias=True)
           (attn_dropout): Dropout(p=0.1, inplace=False)
           (resid_dropout): Dropout(p=0.1, inplace=False)
         )
         (ln_2): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
         (mlp): GPTBigCodeMLP(
           (c_fc): Linear4bit(in_features=6144, out_features=24576, bias=True)
           (c_proj): Linear4bit(in_features=24576, out_features=6144, bias=True)
           (act): GELUActivation()
           (dropout): Dropout(p=0.1, inplace=False)
         )
    

In [71]:
info_kwargs = dict(cfg=cfg, ds_name=ds_name, split_type=split_type)

In [73]:
ds1 = Dataset.from_generator(
    generator=batch_hidden_states,
    info=DatasetInfo(
        description=f"kwargs={info_kwargs}",
        config_name=f,               
    ),
    gen_kwargs=gen_kwargs,
).with_format("numpy")
ds1

Generating train split: 0 examples [00:00, ? examples/s]

get hidden states:   0%|          | 0/10 [00:00<?, ?it/s]

Dataset({
    features: ['hs0', 'scores0', 'hs1', 'scores1', 'ds_index', 'answer', 'question', 'answer_choices', 'template_name', 'label_true', 'label_instructed', 'instructed_to_lie', 'sys_instr_name', 'prompt_truncated'],
    num_rows: 100
})

## Add labels

For our probe. Given next_token scores (logits) we take only the subset the corresponds to our negative tokens (e.g. False, no, ...) and positive tokens (e.g. Yes, yes, affirmative, ...).


In [ ]:
ds1[0]

{'hs0': array([[-19426,  11028, -21389, ..., -21150,   9406,  12613],
        [-18651, -20188, -21171, ..., -22544, -21212,  11559],
        [-17774, -21442, -19704, ..., -20192,  10318,  12760],
        ...,
        [-15081, -15028,  17778, ..., -15613,  17018,  18158],
        [-15197, -14936,  18252, ..., -15201,  17003,  18156],
        [-15041, -15044,  18142, ..., -15195,  15843,  17924]]),
 'scores0': array([ 1.0849609 , -2.5351562 ,  0.62060547, ...,  2.8496094 ,
        -0.7944336 ,  7.4648438 ], dtype=float32),
 'hs1': array([[-19226,   9750, -22246, ..., -20195,  11820,  12634],
        [-18509, -20209, -22402, ..., -21187, -23104,  11082],
        [-17376, -20317, -19616, ..., -19682,   9296,  12591],
        ...,
        [ 14626, -15033,  18443, ..., -15389,  17098,  18489],
        [ 14384, -14750,  18774, ..., -15477,  17123,  18380],
        [ 15284, -14987,  18839, ..., -15650,  17088,  18252]]),
 'scores1': array([ 4.6835938 ,  0.10943604,  4.6054688 , ...,  2.6601562

In [74]:
# class2_ids = choice2ids(tokenizer, r['answer_choices'])
add_txt_ans0 = lambda r: {'txt_ans0': tokenizer.decode(r['scores0'].argmax(-1))}
add_txt_ans1 = lambda r: {'txt_ans1': tokenizer.decode(r['scores1'].argmax(-1))}
add_ans = lambda r: scores2choice_probs(r, choice2ids(tokenizer, r['answer_choices']))

ds3 = (
    ds1
    .map(add_ans)
    .map(add_txt_ans0)
    .map(add_txt_ans1)
)
ds3

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['hs0', 'scores0', 'hs1', 'scores1', 'ds_index', 'answer', 'question', 'answer_choices', 'template_name', 'label_true', 'label_instructed', 'instructed_to_lie', 'sys_instr_name', 'prompt_truncated', 'choice_probs0', 'ans0', 'choice_probs1', 'ans1', 'txt_ans0', 'txt_ans1'],
    num_rows: 100
})

In [75]:
# %debug

## Save to disk

In [76]:
ds3.save_to_disk(f)
f

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

'../.ds/HuggingFaceH4starchat_beta_imdb_train_100'

# QC

In [77]:
ds4 = load_from_disk(f)
ds4

Dataset({
    features: ['hs0', 'scores0', 'hs1', 'scores1', 'ds_index', 'answer', 'question', 'answer_choices', 'template_name', 'label_true', 'label_instructed', 'instructed_to_lie', 'sys_instr_name', 'prompt_truncated', 'choice_probs0', 'ans0', 'choice_probs1', 'ans1', 'txt_ans0', 'txt_ans1'],
    num_rows: 100
})

In [78]:

# QC, check which answers are most common
common_answers = pd.Series(ds4['txt_ans1']).value_counts()
display('Remember it should be binary. Found common LLM answers:', common_answers)

# list unexpected answers
class2choices = dataset_params['choices']
current_choices = set(class2choices[0]+class2choices[1])
unexpected_answers = set(common_answers.head(10).index)-current_choices
if len(unexpected_answers):
    logger.warning(f'found unexpected answers: {unexpected_answers}. You may want to add them to class2choices')
    
mean_prob = ds4['choice_probs1'].sum(-1).mean()
print('mean_prob', mean_prob)
assert ds4['choice_probs1'].sum(-1).mean()>0.4, f"""
Our choices should cover most common answers. But they accounted for a mean probability of {mean_prob:2.2%} (should be >40%). 

To fix this you might want to improve your prompt or add to your choices
"""

'Remember it should be binary. Found common LLM answers:'

0            46
negative     41
Positive      4
positive      2
1             2
\n            2
 =            1
The           1
 positive     1
Name: count, dtype: int64

NameError: name 'dataset_params' is not defined

In [79]:
df = ds2df(ds4)
df

AttributeError: 'DataFrame' object has no attribute 'label'

In [ ]:
# QC check accuracy
# it should manage to lie some of the time when asked to lie. Many models wont lie unless very explicitly asked to, but we don't want to do that, we want to leave some ambiguity in the prompt

d = df.query('lie==True')
acc = (d.desired_ans==d.llm_ans).mean()
print(f"when the model tries to lie... we get this acc {acc:2.2f}")
assert acc>0.1, f"should be acc>0.1 but is acc={acc}"

In [ ]:
# QC by viewing a row
r = ds4[0]
print(r['prompt_truncated'])
print(r['txt_ans1'])

# QC: generation

Let's a quick generation, so we can QC the output and sanity check that the model can actually do the task

In [ ]:
# r = ds[2]
# q = r["prompt_truncated"]

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
# )
# sequences = pipeline(
#     q.lstrip('<|endoftext|>'),
#     max_length=100,
#     do_sample=False,
#     return_full_text=False,
#     eos_token_id=tokenizer.eos_token_id,
# )

# for seq in sequences:
#     print("-" * 80)
#     print(q)
#     print("-" * 80)
#     print(f"`{seq['generated_text']}`")
#     print("-" * 80)
#     print("label", r['label'])


# QC: linear probe

In [ ]:
hs = ds4['hs1']-ds4['hs0']
X = hs.reshape(hs.shape[0], -1)
y = (ds4['ans1'] - ds4['ans0'])>0

In [ ]:
true_switch_sign = ds4['label']*2-1
# true_switch_sign = ds4['true'][:, 0]*2-1
y = ((ds4['ans1'] - ds4['ans0']) * true_switch_sign) > 0

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

# # Define X and y
# X = dm.hs1-dm.hs2
# y = dm.y>0

# split
n = len(y)
max_rows = 1000
print('split size', n//2)
X_train, X_test = X[:n//2], X[n//2:]
y_train, y_test = y[:n//2], y[n//2:]
X_train = X_train[:max_rows]
y_train = y_train[:max_rows]
X_test = X_test[:max_rows]
y_test = y_test[:max_rows]

# scale
scaler = RobustScaler()
scaler.fit(X_train)
X_train2 = scaler.transform(X_train)
X_test2 = scaler.transform(X_test)
print('lr')

lr = LogisticRegression(class_weight="balanced", penalty="l2", max_iter=380)
lr.fit(X_train2, y_train>0)

In [ ]:
print("Logistic cls acc: {:2.2%} [TRAIN]".format(lr.score(X_train2, y_train>0)))
print("Logistic cls acc: {:2.2%} [TEST]".format(lr.score(X_test2, y_test>0)))